In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
wefarm = pd.read_parquet("/content/drive/MyDrive/DataKit_WeFarm/wefarm.parquet")

In [4]:
wefarm["response_sent_dt"] = pd.to_datetime(wefarm["response_sent"], format="ISO8601")
wefarm["question_set_dt"] = pd.to_datetime(wefarm["question_sent"], format="ISO8601")

In [5]:

def parse_datetime_col(df, colstr):
  df[f'{colstr}_year'] = df[colstr].dt.year
  df[f'{colstr}_month'] = df[colstr].dt.month
  df[f'{colstr}_day'] = df[colstr].dt.day
  df[f'{colstr}_hour'] = df[colstr].dt.hour
  df[f'{colstr}_minute'] = df[colstr].dt.minute
  df[f'{colstr}_second'] = df[colstr].dt.second
  df[f'{colstr}_day_name'] = df[colstr].dt.day_name()
  df[f'{colstr}_day_of_week'] = df[colstr].dt.day_of_week
  df[f'{colstr}_month_name'] = df[colstr].dt.month_name()
  df[f'{colstr}_quarter'] = df[colstr].dt.quarter
  df[f'{colstr}_week'] = df[colstr].dt.isocalendar().week
  df[f'{colstr}_date'] = df[colstr].dt.date
  df[f'{colstr}_time'] = df[colstr].dt.time

  return df

In [6]:
wefarm = parse_datetime_col(wefarm, "response_sent_dt")
wefarm = parse_datetime_col(wefarm, "question_set_dt")

In [7]:
niche = pd.read_parquet("/content/drive/MyDrive/DataKit_WeFarm/en_questions_cat_niche.parquet")
broad = pd.read_parquet("/content/drive/MyDrive/DataKit_WeFarm/en_questions_cat_broad.parquet")

In [8]:
all = pd.merge(wefarm, niche, on="question_id").merge(
  broad, on="question_id"
)

In [9]:
import json

with open("/content/drive/MyDrive/DataKit_WeFarm/segment_user_ids.json", "r") as r:
    segment_user_id_dct = json.load(r)

In [10]:
segment_user_id_dct.keys()

dict_keys(['user_activity_post_count', 'speed_post_response', 'unique_askers', 'tenure'])

In [11]:
# segment_user_id_dct["user_activity_post_count"].keys()

In [12]:
from collections import defaultdict

user_segments = defaultdict(dict)
for agg, dct in segment_user_id_dct.items():
  for segment, user_ids in dct.items():
    for user_id in user_ids:
      user_segments[user_id][agg] = segment

row_lst = []
for user_id, segments in user_segments.items():
  row = {"user_id": user_id}
  row.update(segments)
  row_lst.append(row)

user_segments_df = pd.DataFrame(row_lst)

In [13]:
all1 = all.merge(user_segments_df, left_on="question_user_id", right_on="user_id", how="inner")

In [14]:
all2 = all1[all1["question_user_country_code"].isin(["ug", "ke"])]

In [15]:
all3 = all2.drop_duplicates("question_id")

In [16]:
all3["question_user_country_code"] = [j.upper() for j in all3["question_user_country_code"]]

/tmp/ipython-input-948224471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all3["question_user_country_code"] = [j.upper() for j in all3["question_user_country_code"]]


In [17]:
import pickle

filename="/content/drive/MyDrive/DataKit_WeFarm/Dashboard/intersections/segmentation_intersections_all.pkl"
with open(filename, "rb") as rb:
  intersection_dct = pickle.load(rb)

In [18]:
wefarm_cleaned=all3

# define topics lists

In [19]:
wefarm_cleaned.iloc[:5, 54:173]

,financial-inclusion_price,financial-inclusion_sell,financial-inclusion_buy,diversification_obtain,diversification_acquire,question_what,question_when,question_how,question_which,crop_harvest,climate_time,disease_harvest,livestock_livestock,livestock_animals,livestock_sheep,livestock_cattle,livestock_donkey,livestock_pig,livestock_cow,livestock_goat,livestock_camel,livestock_hen,livestock_chicken,livestock_poultry,livestock_bee,diversification_expand,diversification_begin,crop_plant,crop_variety,disease_plant,crop_seed,crop_grow,crop_mulch,diversification_grow,crop_crop,crop_potatoes,crop_passion,crop_maize,crop_banana,crop_coffee,crop_tea,crop_onions,crop_bean,crop_cabbage,climate_rain,diversification_add,question_who,disease_rabbit,diversification_use,crop_tomatoes,crop_rice,disease_disease,disease_care,crop_keep,disease_keep,financial-inclusion_market,financial-inclusion_cost,question_many,crop_layer,disease_treat,disease_prevent,disease_chemical,crop_fruit,livestock_milk,crop_leave,crop_land,disease_leave,livestock_egg,livestock_breed,crop_manure,diversification_increase,climate_season,disease_control,climate_sun,crop_yield,livestock_dairy,crop_soil,diversification_irrigation,diversification_scale,financial-inclusion_money,question_where,diversification_best,disease_spray,disease_turn,disease_attack,financial-inclusion_fee,crop_type,climate_climate,disease_medicine,disease_tick,disease_weed,disease_pests,financial-inclusion_finance,financial-inclusion_investment,financial-inclusion_loan,diversification_new,livestock_lay,climate_heat,climate_drought,diversification_clear,disease_black,disease_space,crop_fertilizer,disease_sick,climate_temperature,climate_flood,disease_affect,financial-inclusion_sum,disease_unhealthy,financial-inclusion_bank,financial-inclusion_insurance,disease_mean,financial-inclusion_credit,diversification_acreage,diversification_diversify,diversification_register,climate_water,diversification_shift,financial-inclusion_economy
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN

In [20]:
niche_collst = wefarm_cleaned.columns[54:173]

In [21]:
wefarm_cleaned.iloc[:5, -12:-5]

,financial-inclusion,diversification,question,crop,climate,disease,livestock
0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
1,NaN,NaN,1.0,1.0,1.0,1.0,1.0
2,NaN,1.0,1.0,1.0,NaN,1.0,NaN
4,1.0,1.0,1.0,1.0,1.0,1.0,NaN
8,NaN,1.0,1.0,1.0,NaN,1.0,1.0


In [22]:
broad_collst = wefarm_cleaned.columns[-12:-5]

In [23]:
time_collst = [
  "question_set_dt_year",
  "question_set_dt_month",
  "question_set_dt_hour",
  "question_set_dt_day_of_week"
]

# testing out drop-in function from claude (based on my previous groupby code)

In [24]:
from tqdm import tqdm
import pickle
import json
import numpy as np

def precompute_all_intersection_groupbys(df, intersection_dict, broad_collst, niche_collst):
    """Precompute ALL groupbys for all intersections - broad, niche, and time with SE"""

    all_groupbys = {
        'broad': {},
        'niche': {},
        'time_broad': {},
        'time_niche': {},
        "counts": {}
    }

    time_dims = [
        "question_set_dt_year",
        "question_set_dt_month",
        "question_set_dt_hour",
        "question_set_dt_day_of_week"
    ]

    for combo_key, user_indices in tqdm(intersection_dict.items(), desc="Precomputing groupbys"):
        # Filter to this intersection
        filtered_df = df.loc[user_indices]

        # 1. Counts by country
        country_counts = filtered_df.groupby('question_user_country_code').size().to_dict()
        all_groupbys['counts'][combo_key] = {
            'total': len(filtered_df),
            'by_country': country_counts
        }

        # 2. Broad topics by country with SE
        gb_sum = filtered_df.groupby("question_user_country_code")[broad_collst].sum()
        gb_count = filtered_df.groupby("question_user_country_code").size()

        row_sums = gb_sum[broad_collst].sum(axis=1)
        gb_pct = gb_sum[broad_collst].div(row_sums, axis=0)
        gb_se = np.sqrt(gb_pct * (1 - gb_pct)).div(np.sqrt(gb_count), axis=0)

        gb_pct = gb_pct.reset_index()
        gb_se = gb_se.reset_index()
        gb_sum = gb_sum.reset_index()
        gb_sum['count'] = gb_count.values

        gb_pct_long = gb_pct.melt(
            id_vars=["question_user_country_code"],
            value_vars=broad_collst,
            var_name="category",
            value_name="pct"
        )

        gb_se_long = gb_se.melt(
            id_vars=["question_user_country_code"],
            value_vars=broad_collst,
            var_name="category",
            value_name="se"
        )

        gb_broad_long = gb_pct_long.merge(
            gb_se_long,
            on=["question_user_country_code", "category"]
        ).merge(
            gb_sum[["question_user_country_code", "count"]],
            on=["question_user_country_code"]
        ).sort_values(by="category")

        all_groupbys['broad'][combo_key] = gb_broad_long

        # 3. Niche topics by country with SE
        gb_sum = filtered_df.groupby("question_user_country_code")[niche_collst].sum()
        gb_count = filtered_df.groupby("question_user_country_code").size()

        row_sums = gb_sum[niche_collst].sum(axis=1)
        gb_pct = gb_sum[niche_collst].div(row_sums, axis=0)
        gb_se = np.sqrt(gb_pct * (1 - gb_pct)).div(np.sqrt(gb_count), axis=0)

        gb_pct = gb_pct.reset_index()
        gb_se = gb_se.reset_index()
        gb_sum = gb_sum.reset_index()
        gb_sum['count'] = gb_count.values

        gb_pct_long = gb_pct.melt(
            id_vars=["question_user_country_code"],
            value_vars=niche_collst,
            var_name="category",
            value_name="pct"
        )

        gb_se_long = gb_se.melt(
            id_vars=["question_user_country_code"],
            value_vars=niche_collst,
            var_name="category",
            value_name="se"
        )

        gb_niche_long = gb_pct_long.merge(
            gb_se_long,
            on=["question_user_country_code", "category"]
        ).merge(
            gb_sum[["question_user_country_code", "count"]],
            on=["question_user_country_code"]
        )

        gb_niche_long["broad_type"] = gb_niche_long["category"].str.split("_").str[0]
        gb_niche_long["niche"] = gb_niche_long["category"].str.split("_").str[1]
        gb_niche_long = gb_niche_long.sort_values(
            by=["broad_type", "pct", "niche"],
            ascending=[True, False, True]
        )

        all_groupbys['niche'][combo_key] = gb_niche_long

        # 4. Time-based groupbys (broad) with SE
        all_groupbys['time_broad'][combo_key] = {}
        for time_dim in time_dims:
            gb_sum = filtered_df.groupby([
                "question_user_country_code",
                time_dim
            ])[broad_collst].sum()

            gb_count = filtered_df.groupby([
                "question_user_country_code",
                time_dim
            ]).size()

            row_sums = gb_sum[broad_collst].sum(axis=1)
            gb_pct = gb_sum[broad_collst].div(row_sums, axis=0)
            gb_se = np.sqrt(gb_pct * (1 - gb_pct)).div(np.sqrt(gb_count), axis=0)

            gb_pct = gb_pct.reset_index()
            gb_se = gb_se.reset_index()
            gb_sum = gb_sum.reset_index()
            gb_sum['count'] = gb_count.values

            gb_pct_long = gb_pct.melt(
                id_vars=["question_user_country_code", time_dim],
                value_vars=broad_collst,
                var_name="category",
                value_name="pct"
            )

            gb_se_long = gb_se.melt(
                id_vars=["question_user_country_code", time_dim],
                value_vars=broad_collst,
                var_name="category",
                value_name="se"
            )

            gb_time_broad_long = gb_pct_long.merge(
                gb_se_long,
                on=["question_user_country_code", time_dim, "category"]
            ).merge(
                gb_sum[["question_user_country_code", time_dim, "count"]],
                on=["question_user_country_code", time_dim]
            ).sort_values(by=time_dim)

            all_groupbys['time_broad'][combo_key][time_dim] = gb_time_broad_long

        # 5. Time-based groupbys (niche) with SE
        all_groupbys['time_niche'][combo_key] = {}
        for time_dim in time_dims:
            gb_sum = filtered_df.groupby([
                "question_user_country_code",
                time_dim
            ])[niche_collst].sum()

            gb_count = filtered_df.groupby([
                "question_user_country_code",
                time_dim
            ]).size()

            row_sums = gb_sum[niche_collst].sum(axis=1)
            gb_pct = gb_sum[niche_collst].div(row_sums, axis=0)
            gb_se = np.sqrt(gb_pct * (1 - gb_pct)).div(np.sqrt(gb_count), axis=0)

            gb_pct = gb_pct.reset_index()
            gb_se = gb_se.reset_index()
            gb_sum = gb_sum.reset_index()
            gb_sum['count'] = gb_count.values

            gb_pct_long = gb_pct.melt(
                id_vars=["question_user_country_code", time_dim],
                value_vars=niche_collst,
                var_name="category",
                value_name="pct"
            )

            gb_se_long = gb_se.melt(
                id_vars=["question_user_country_code", time_dim],
                value_vars=niche_collst,
                var_name="category",
                value_name="se"
            )

            gb_time_niche_long = gb_pct_long.merge(
                gb_se_long,
                on=["question_user_country_code", time_dim, "category"]
            ).merge(
                gb_sum[["question_user_country_code", time_dim, "count"]],
                on=["question_user_country_code", time_dim]
            ).sort_values(
                by=time_dim,
                ascending=False
            )

            gb_time_niche_long["broad_type"] = gb_time_niche_long["category"].str.split("_").str[0]
            gb_time_niche_long["niche"] = gb_time_niche_long["category"].str.split("_").str[1]

            all_groupbys['time_niche'][combo_key][time_dim] = gb_time_niche_long

    return all_groupbys

# Run preprocessing
all_groupbys = precompute_all_intersection_groupbys(
    wefarm_cleaned,
    intersection_dct,
    broad_collst,
    niche_collst
)

Precomputing groupbys: 100%|██████████| 1296/1296 [05:51<00:00,  3.69it/s]


In [25]:
list(all_groupbys["counts"].values())[2]

{'total': 135823, 'by_country': {'KE': 78783, 'UG': 57040}}

In [26]:
type(all_groupbys["broad"])

dict

In [27]:
cs_cs = ["1", "2", "3", "4", "5"]
spr_cs = ["1", "2", "3", "4", "5"]
ten_cs = ["1", "2", "3", "4", "5"]
ua_cs = ["1", "2", "3", "4", "5"]

In [28]:
combo_dct = {
  "unique_askers": cs_cs,
  "speed_post_response": spr_cs,
  "tenure": ten_cs,
  "user_activity_post_count": ua_cs
}

key_json = json.dumps(combo_dct, sort_keys=True)

In [29]:
all_groupbys["broad"][key_json]

,question_user_country_code,category,pct,se,count
8,KE,climate,0.057200,0.000192,1461680
9,UG,climate,0.047399,0.000199,1144162
6,KE,crop,0.189321,0.000324,1461680
7,UG,crop,0.201492,0.000375,1144162
10,KE,disease,0.172799,0.000313,1461680
11,UG,disease,0.179404,0.000359,1144162
2,KE,diversification,0.131267,0.000279,1461680
3,UG,diversification,0.126707,0.000311,1144162
0,KE,financial-inclusion,0.080284,0.000225,1461680
1,UG,financial-inclusion,0.088468,0.000265,1144162


In [30]:
list(all_groupbys["broad"].values())[0]

,question_user_country_code,category,pct,se,count
8,KE,climate,0.057200,0.000192,1461680
9,UG,climate,0.047399,0.000199,1144162
6,KE,crop,0.189321,0.000324,1461680
7,UG,crop,0.201492,0.000375,1144162
10,KE,disease,0.172799,0.000313,1461680
11,UG,disease,0.179404,0.000359,1144162
2,KE,diversification,0.131267,0.000279,1461680
3,UG,diversification,0.126707,0.000311,1144162
0,KE,financial-inclusion,0.080284,0.000225,1461680
1,UG,financial-inclusion,0.088468,0.000265,1144162


In [31]:
all_groupbys.keys()

dict_keys(['broad', 'niche', 'time_broad', 'time_niche', 'counts'])

In [32]:
# with open('groupby_segmentation_intersection_all.pkl', 'wb') as f:
#     pickle.dump(all_groupbys, f)

# !cp groupby_segmentation_intersection_all.pkl /content/drive/MyDrive/DataKit_WeFarm/Dashboard/intersections/

In [33]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 225.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.6 MB/s eta 0:00:00


In [35]:
# import boto3

# with open("intersection_groupbys.pkl", "wb") as wb:
#   pickle.dump(all_groupbys, wb)

# s3 = boto3.client("s3")

# s3.upload_file(
#   Filename="intersection_groupbys.pkl",
#   Bucket="datakit-farmers-dashboard",
#   Key="intersection_groupbys.pkl"
# )

In [36]:
import gzip
with gzip.open('intersection_groupbys_complete.pkl.gz', 'wb') as wb:
    pickle.dump(all_groupbys, wb)

In [37]:
s3 = boto3.client("s3")

s3.upload_file(
    Filename="intersection_groupbys_complete.pkl.gz",
    Bucket="datakit-farmers-dashboard",
    Key="intersection_groupbys_complete.pkl.gz"
)